In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# Load trained model
model = tf.keras.models.load_model("spice_classifier.h5")

# Manually extract class labels from the dataset folder
dataset_path = "dataset/train"  # Path to your train directory
class_labels = sorted(os.listdir(dataset_path))  # Sorted to match class_indices order

# Load and preprocess the image
img_path = "sample6.jpg"  # Path to your test image
img_size = (224, 224)  # Same size as used during training

img = image.load_img(img_path, target_size=img_size)  # Load image
img_array = image.img_to_array(img)  # Convert to array
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array /= 255.0  # Normalize (same as during training)

# Predict class probabilities
predictions = model.predict(img_array)[0]  # Get probability array

# Print all spices with confidence scores
print("Predictions:")
for i, prob in enumerate(predictions):
    print(f"{class_labels[i]}: {prob*100:.2f}%")

# Print top spices detected
threshold = 0.10  # 10% confidence threshold
detected_spices = [(class_labels[i], prob) for i, prob in enumerate(predictions) if prob >= threshold]
detected_spices.sort(key=lambda x: x[1], reverse=True)  # Sort by confidence

print("\nDetected Spices:")
for spice, confidence in detected_spices:
    print(f"{spice}: {confidence*100:.2f}%")


In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

# Load trained model
model = tf.keras.models.load_model("spice_classifier.h5")

# Define image size (must match model input size)
img_size = (224, 224)

# Load class labels from dataset folder
dataset_path = "dataset/train"
class_labels = sorted(os.listdir(dataset_path))

# Load test images and true labels
test_dir = "dataset/test"  # Ensure you have a test dataset
test_images = []
true_labels = []

for spice_class in os.listdir(test_dir):
    spice_class_path = os.path.join(test_dir, spice_class)
    if os.path.isdir(spice_class_path):  # Ensure it's a directory
        for img_name in os.listdir(spice_class_path):
            img_path = os.path.join(spice_class_path, img_name)
            image = cv2.imread(img_path)
            if image is not None:
                resized_image = cv2.resize(image, img_size)
                test_images.append(resized_image / 255.0)
                true_labels.append(class_labels.index(spice_class))  # Store index of true label

# Convert lists to numpy arrays
test_images = np.array(test_images)
true_labels = np.array(true_labels)

# Predict class indices
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix
plt.figure(figsize=(12, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix for Spice Classification")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.show()

# Print classification report (precision, recall, F1-score)
print("Classification Report:\n", classification_report(true_labels, predicted_labels, target_names=class_labels))
